# Labelling Algorithm
The intention of this notebook is, to learn how to use C Code in Python and with that knowledge implement a labelling algorithm for the CVRP in C.

## CFFI Tutorial
https://medium.com/@torbengraebergt/c-c-and-python-class-interfacing-f45a9ee352f4

In [9]:
from cffi import FFI

dllURI = "cpp_lib.so"
funDef = "unsigned int myProcessorInit(int exp_in, int size_in);"

ffi = FFI()
cpp_lib = ffi.dlopen(dllURI)
ffi.cdef(funDef)

# result = c_lib.myProcessorInit(2,10)

In [12]:
def extract_function_def(headerFileURI):
    funDef = ''
    with open(headerFileURI) as headerFile:
        hf = headerFile.read()
        idxLo = hf.find(r'extern "C" {')
        idxHi = hf.find('} //end extern "C"')
        while (idxLo>0 and idxHi>0):
            tmp = hf[idxLo+12:idxHi]
            if "#" not in tmp:
                funDef += tmp
            hf = hf[idxHi+18:]
            idxLo = hf.find(r'extern "C" {')
            idxHi = hf.find('} //end extern "C"')
    return funDef


def ensure_folder_or_file(fileURI):
    if not os.path.isdir(fileURI) and not os.path.isfile(fileURI):
        raise OSError("Target does not exist.\nSearched for: " + fileURI + "\n")
        
functionDefinitions = "unsigned int myProcessorInit(int exp_in, int size_in);\n    int myProcessorProcess(unsigned int id, double *d, int size);"
ffi.cdef(functionDefinitions, override=True)

In [29]:
class myProcessor():

    def __init__(self, exp, size):
        self.id = cpp_lib.myProcessorInit(exp, size)

    def process(self, data, size):
        data = ensure_np_type(data, 'float64')
        pointer_to_data = get_pointer_to_np_arr(data, "double*", ffi)
        
        ret = cpp_lib.myProcessorProcess(self.id, pointer_to_data, size)
        if ret<0:
            raise ValueError("Sth bad happened in c++ code")
        else:
            return data

In [30]:
import numpy as np

def ensure_np_type(arr, target_type):
    if arr.dtype is not np.dtype(target_type):
        return arr.astype(np.dtype(target_type))
    else:
        return arr

def get_pointer_to_np_arr(arr, ctype, ffi):
    return ffi.cast(ctype, arr.ctypes.data)

In [31]:
processor = myProcessor(2,10)
data = np.arange(10)
processor.process(data,10)

array([ 0.,  1.,  4.,  9., 16., 25., 36., 49., 64., 81.])